In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb

# Load dataset
df = pd.read_csv("Final_dataset+20000.csv")  # Replace with actual path

# Data preprocessing
X = df.drop(columns=['Label'])  # Features
y = df['Label']  # Target

# Encode target labels (for multiclass classification)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Optional: Handle class imbalance (XGBoost handles imbalance well, so this step is optional)
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize XGBoost classifier
xgb_model = xgb.XGBClassifier(
    objective='multi:softmax',          # Multiclass classification
    num_class=len(np.unique(y)),        # Number of classes
    eval_metric='mlogloss',
    use_label_encoder=False,
    seed=42
)

# Train the XGBoost model
xgb_model.fit(X_train, y_train)

# Predict on the test data
y_pred = xgb_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"\n✅ XGBoost Accuracy: {accuracy * 100:.2f}%")
print("✅ Classification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Save the model
xgb_model.save_model("ddos_xgboost_model.json")
print("\n📁 XGBoost model saved as 'ddos_xgboost_model.json'")


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [03:49:47] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



✅ XGBoost Accuracy: 83.37%
✅ Classification Report:
                precision    recall  f1-score   support

       BENIGN       1.00      1.00      1.00     60869
    DrDoS_DNS       0.76      0.27      0.40     59959
   DrDoS_LDAP       0.50      0.66      0.57     59953
  DrDoS_MSSQL       0.92      0.95      0.93     59978
    DrDoS_NTP       0.99      1.00      0.99     59297
DrDoS_NetBIOS       0.95      0.98      0.97     59978
   DrDoS_SNMP       0.66      0.84      0.74     59985
   DrDoS_SSDP       0.99      0.96      0.97     59981

     accuracy                           0.83    480000
    macro avg       0.85      0.83      0.82    480000
 weighted avg       0.85      0.83      0.82    480000


📁 XGBoost model saved as 'ddos_xgboost_model.json'


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, classification_report

# Load dataset
df = pd.read_csv(r"Final_dataset+20000.csv")

# Features and labels
X = df.drop(columns=['Label'])
y = df['Label']

# Label encoding and one-hot conversion
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_encoded), y=y_encoded)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, stratify=y_encoded, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape for LSTM [samples, timesteps, features]
X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

# Build LSTM model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(X_train_lstm.shape[1], 1)),
    Dropout(0.3),
    LSTM(32),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dropout(0.2),
    Dense(y_categorical.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train_lstm, y_train, validation_data=(X_test_lstm, y_test), epochs=30, batch_size=32, class_weight=class_weights_dict)

# Predict and evaluate
y_pred_probs = model.predict(X_test_lstm)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

print("Accuracy:", accuracy_score(y_true, y_pred))
print("Classification Report:\n", classification_report(y_true, y_pred, target_names=label_encoder.classes_))

# Save model
model.save("ddos_lstm_multiclass_model.h5")


C:\Users\meera\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
41500/41500 ━━━━━━━━━━━━━━━━━━━━ 514s 12ms/step - accuracy: 0.7066 - loss: 0.7304 - val_accuracy: 0.7647 - val_loss: 0.5313
Epoch 2/30
41500/41500 ━━━━━━━━━━━━━━━━━━━━ 566s 14ms/step - accuracy: 0.7651 - loss: 0.5438 - val_accuracy: 0.7742 - val_loss: 0.5113
Epoch 3/30
41500/41500 ━━━━━━━━━━━━━━━━━━━━ 915s 22ms/step - accuracy: 0.7704 - loss: 0.5275 - val_accuracy: 0.7798 - val_loss: 0.5049
Epoch 4/30
41500/41500 ━━━━━━━━━━━━━━━━━━━━ 1915s 46ms/step - accuracy: 0.7760 - loss: 0.5185 - val_accuracy: 0.7846 - val_loss: 0.4971
Epoch 5/30
41500/41500 ━━━━━━━━━━━━━━━━━━━━ 1329s 32ms/step - accuracy: 0.7805 - loss: 0.5121 - val_accuracy: 0.7800 - val_loss: 0.5001
Epoch 6/30
41500/41500 ━━━━━━━━━━━━━━━━━━━━ 523s 13ms/step - accuracy: 0.7828 - loss: 0.5063 - val_accuracy: 0.7932 - val_loss: 0.4931
Epoch 7/30
41500/41500 ━━━━━━━━━━━━━━━━━━━━ 563s 14ms/step - accuracy: 0.7832 - loss: 0.5033 - val_accuracy: 0.7934 - val_loss: 0.4881
Epoch 8/30
41500/41500 ━━━━━━━━━━━━━━━━━━━━ 570s 14ms